In [11]:
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import models
from tqdm.notebook import tqdm

from task_dataset import load_dataset, TaskDataset, shuffle  # noqa !!! don't remove this line
from adversary import fgsm_attack, pgd_attack

In [12]:
# visualization of the dataset is in dataset.ipynb
dataset = load_dataset('Train.pt')
num_classes = len(set(dataset.labels))

# shuffle the dataset
# shuffle(dataset)

train_size = int(0.8 * len(dataset))
test_size = int(0.1 * len(dataset))
dev_size = len(dataset) - train_size - test_size

# TODO: czy to na pewno poprawnie dziala?
train_dataset, test_dataset, dev_dataset = torch.utils.data.random_split(
    dataset, [train_size, test_size, dev_size])

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=0)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=0)
dev_loader = DataLoader(dev_dataset, batch_size=1, shuffle=False, num_workers=0)  # FIXME: batch_size=1

/Users/igorjakus/projects/ensemble-ai/task_3/task_dataset.py:33: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  dataset = torch.load(path)


In [13]:
# Create model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cpu


In [14]:
class Resnet18(nn.Module):
    def __init__(self):
        super(Resnet18, self).__init__()
        self.model = models.resnet18(pretrained=True)
        self.model_name = "resnet18"
        self.model.fc = nn.Linear(self.model.fc.in_features, num_classes)
        self.model = self.model.to(device)

        self.optimizer = optim.Adam(self.model.parameters(), lr=0.001)
        self.criterion = nn.CrossEntropyLoss()

    def forward(self, x):
        return self.model(x)

In [ ]:
def train(model: Resnet18, dataloader: DataLoader, num_epochs: int):
    # hyperparameters
    learning_rate = 0.001
    epsilon = 0.1
    alpha = 0.01
    iters = 10
    
    model.train()  # make sure the model is in training mode
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    for epoch in range(num_epochs):
        for batch_idx, batch in enumerate(dataloader):
            idxs, imgs, labels = batch

            # Standard forward pass
            outputs = model(imgs)
            loss = criterion(outputs, labels)
            
            # FGSM Attack
            adv_inputs = fgsm_attack(model, criterion, imgs, labels, epsilon)
            adv_outputs = model(adv_inputs)
            adv_loss = criterion(adv_outputs, labels)
            
            # PGD Attack
            pgd_inputs = pgd_attack(model, criterion, imgs, labels, epsilon, alpha, iters)
            pgd_outputs = model(pgd_inputs)
            pgd_loss = criterion(pgd_outputs, labels)
            
            # Total loss
            total_loss = (loss + adv_loss + pgd_loss) / 3
            
            optimizer.zero_grad()
            total_loss.backward()
            optimizer.step()

            if batch_idx % 10 == 0:
                print(f"Epoch [{epoch+1}/{num_epochs}], Step [{batch_idx+1}/{len(dataloader)}], Loss: {total_loss.item():.4f}")
    
    print("Training finished!")

In [16]:
model = Resnet18()

/Users/igorjakus/.pyenv/versions/3.12.4/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/igorjakus/.pyenv/versions/3.12.4/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [17]:
train(model, dev_loader, 1)

Epoch [1/1], Step [1/1250], Loss: 1.5004
Epoch [1/1], Step [11/1250], Loss: 0.8908
Epoch [1/1], Step [21/1250], Loss: 0.6449
Epoch [1/1], Step [31/1250], Loss: 1.7480
Epoch [1/1], Step [41/1250], Loss: 0.9551
Epoch [1/1], Step [51/1250], Loss: 1.7237
Epoch [1/1], Step [61/1250], Loss: 1.0367
Epoch [1/1], Step [71/1250], Loss: 3.3167
Epoch [1/1], Step [81/1250], Loss: 1.1122
Epoch [1/1], Step [91/1250], Loss: 0.8689
Epoch [1/1], Step [101/1250], Loss: 2.6438
Epoch [1/1], Step [111/1250], Loss: 2.2765
Epoch [1/1], Step [121/1250], Loss: 1.3093
Epoch [1/1], Step [131/1250], Loss: 1.3777
Epoch [1/1], Step [141/1250], Loss: 1.5192
Epoch [1/1], Step [151/1250], Loss: 0.6847
Epoch [1/1], Step [161/1250], Loss: 3.0586
Epoch [1/1], Step [171/1250], Loss: 1.1239
Epoch [1/1], Step [181/1250], Loss: 1.0968
Epoch [1/1], Step [191/1250], Loss: 1.0864
Epoch [1/1], Step [201/1250], Loss: 2.2855
Epoch [1/1], Step [211/1250], Loss: 4.6864
Epoch [1/1], Step [221/1250], Loss: 1.7319
Epoch [1/1], Step [231

KeyboardInterrupt: 

In [18]:
def evaluate(model: Resnet18, dataloader: DataLoader):
    model.eval()  # make sure model is in evaluation mode
    with torch.no_grad():
        # clean accuracy
        correct, total = 0, 0
        for batch in dataloader:
            idxs, imgs, labels = batch
            outputs = model(imgs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
        accuracy_clean = 100 * correct / total
        print(f"{accuracy_clean=}")

        # FGSM accuracy
        correct, total = 0, 0
        for batch in dataloader:
            idxs, imgs, labels = batch
            adv_inputs = fgsm_attack(model, model.criterion, imgs, labels, 0.1)
            outputs = model(adv_inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
        accuracy_fgsm = 100 * correct / total
        print(f"{accuracy_fgsm=}")

        # PGD accuracy
        correct, total = 0, 0
        for batch in dataloader:
            idxs, imgs, labels = batch
            adv_inputs = pgd_attack(model, model.criterion, imgs, labels, 0.1, 0.01, 10)
            outputs = model(adv_inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
        accuracy_pgd = 100 * correct / total
        print(f"{accuracy_pgd=}")

In [19]:
evaluate(model, test_loader)

accuracy_clean=25.12


RuntimeError: element 0 of tensors does not require grad and does not have a grad_fn